In [1]:
import os
import glob
import datetime
from datetime import timezone as timezone
import time

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

from selenium import webdriver
import re
import requests
from multiprocessing import Pool
import imp
import support
imp.reload(support)
from support import download_links
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
location={'monterrey':{'lon':-100.316673,'lat':25.66667}}
norm_data={'monterrey':{'mean':{'Total':1066.444949,'Rate':18.983134,
                               'Ending':54.022061,'feels_like':295.112323,
                               'pressure':1016.545350,'humidity':67.419076,
                               'Day_sin':-0.000163,'Day_cos':0.000212,
                               'Week_sin':0.001562,'Week_cos':-0.001023,
                               'Wx':-0.070782,'Wy':1.287389}
                        ,'std':{'Total':638.168254,'Rate':0.882050,
                               'Ending':9.875657,'feels_like':8.362288,
                               'pressure':5.600516,'humidity':20.231055,
                               'Day_sin':0.707119,'Day_cos':0.707119,
                               'Week_sin':0.707117,'Week_cos':0.707118,
                               'Wx':1.513531,'Wy':1.864332}}}

In [3]:
def geteprice():
    try:
        !mkdir data
    except:
        pass
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--window-size=1920,1080")
    chromeOptions.add_argument("--start-maximized")
    chromeOptions.add_argument("--headless")
    driver=webdriver.Chrome(options=chromeOptions) #
    driver.get('https://www.cenace.gob.mx/Paginas/SIM/Reportes/H_PreciosEnergiaSisMEM.aspx?N=26&opc=divCssPreEnergia&site=Precios%20de%20la%20energ%C3%ADa/Precios%20de%20Nodos%20Distribuidos/MDA/Diarios&tipoArch=C&tipoUni=SIN&tipo=Diarios&nombrenodop=Precios%20de%20Nodos%20Distribuidos')
    path1='//*[@id="ctl00_ContentPlaceHolder1_treePrincipal"]/ul/li[1]/ul/li[1]/div/span[2]'
    path2='//*[@id="ctl00_ContentPlaceHolder1_treePrincipal"]/ul/li[1]/ul/li[2]/div/span[2]'
    path3='//*[@id="products"]/tbody/tr[2]/td[2]/table/tbody/tr[8]/td[2]'
    try:
        element = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, path2))
        )
    except:
        print('Time over for electricity price.')
    check1=driver.find_element(By.XPATH, path1)
    check2=driver.find_element(By.XPATH, path2)
    check1.click()
    check2.click()
    try:
        element = WebDriverWait(driver, 60).until(
        EC.presence_of_element_located((By.XPATH, path3))
    )
    except:
        print('Time over for electricity price table contents.')
    soup = BeautifulSoup(driver.page_source, "lxml")
    table=soup.find('table',id='products').find('tbody').find_all('tr',{"class":"orders"})
    links_total=[]
    csv_re=".csv"
    SIN_re='_SIN_'
    for order in table:
        try:
            for tr in order.find('tbody').find_all('tr'):
                tds=tr.find_all('td')
                try:
                    date=tds[0].text.strip().replace('/','-')
                    link=tds[1].find('a')['href'].replace('../../..','https://www.cenace.gob.mx')
                    if re.search(csv_re,link)!=None and re.search(SIN_re,link)!=None:
                        links_total.append((date,link))
                except:
                    pass
        except:
            pass
    
    p=Pool(4)
    p.starmap(download_links,links_total)
    p.close()
    p.join()
    driver.quit()
    
    

In [4]:
#Helper function to determin the starting row
def starting_row(fname):
    with open(fname) as f:
        for i,l in enumerate(f):
            if 'Hora' in l:
                return i+1

#Timestamp convertion
def TS_convert(hour):
    global day,month,year
    hour=int(hour)-1
    if hour<24:
        time=datetime.datetime(year=year,month=month,day=day,hour=hour)
        return time
    else:
        return None
day,month,year=0,0,0
def generate_pricepd():
    files=glob.glob('./data/*')
    rule='(20\d\d)-(\d\d)-(\d\d)'
    #Initialize full dataset
    dataset_full=pd.DataFrame()

    #initialize global variables
    global day,month,year

    for file in files:
        year,month,day=re.findall(rule,file)[0]
        day=int(day)
        month=int(month)
        year=int(year)
        temp_df=pd.read_csv(file,skip_blank_lines=False,skiprows=starting_row(file),names=['Hour','Region','Total','Energy','Lost','Congestion'])
        temp_df['Timestamp']=temp_df['Hour'].apply(TS_convert)
        temp_df['Date']=datetime.datetime(year=year,month=month,day=day).date()
        temp_df=temp_df[temp_df['Timestamp'].notnull()]
        dataset_full=pd.concat([dataset_full,temp_df])
        dataset_full=dataset_full.sort_values(by=['Region','Timestamp'])
    return dataset_full

In [5]:
# Scraping Crude oil price
def get_crude(end):
    today = (datetime.date.today()-datetime.timedelta(days=1))
    start = today-datetime.timedelta(days=30)
    url='https://cn.investing.com/commodities/crude-oil-historical-data'
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--window-size=1920,1080")
    chromeOptions.add_argument("--start-maximized")
    chromeOptions.add_argument("--headless")

    
    driver=webdriver.Chrome(options=chromeOptions)#
    driver.get(url)
    year=start.year
    month=start.month
    day=start.day
    sdate=str(year)+'/'+str(month)+'/'+str(day)
    WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="widgetFieldDateRange"]'))).click()
    element = driver.find_element_by_id("startDate")
    element.clear()
    element.send_keys(sdate)
    WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="applyBtn"]'))).click()
    try:
        element = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="curr_table"]/tbody/tr[1]/td[1]'))
        )
    except:
        print('No Table loaded')
    soup = BeautifulSoup(driver.page_source, "lxml")
    table=soup.find_all('tbody')[1].find_all('tr')
    crude_oil_data=[]

    for tr in table:
        temp=tr.find_all('td')
        date=datetime.datetime.fromtimestamp(float(temp[0].attrs['data-real-value'])).date()
        price=float(temp[1].attrs['data-real-value'])
        crude_oil_data.append([date,price])
    headings=['Date','Ending']
    Crude_df=pd.DataFrame(data=crude_oil_data,columns=headings)
    #noticed some days doesn't have a price. Taking the prior day price
    mindate=Crude_df['Date'].min()
    
    index=mindate
    while index <= end:
        if len(Crude_df[Crude_df['Date']==index])==0:
            prior=index-datetime.timedelta(days=1)
            price=float(Crude_df[Crude_df['Date']==prior]['Ending'])
            temp=pd.DataFrame(data=[[index,price]],columns=headings)
            Crude_df=pd.concat([Crude_df,temp])
        index+=datetime.timedelta(days=1)
    Crude_df=Crude_df.sort_values('Date').reset_index()[['Date','Ending']]
    driver.quit()
    return Crude_df


In [6]:
def get_exchange(end):
    url='https://www.currency-converter.org.uk/currency-rates/historical/table/USD-MXN.html'
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content, "lxml")
    date_re='(\d+)/(\d+)/(\d\d\d\d)'
    table=soup.find_all('table')[0].find_all('tr')[1:]
    exchange_data=[]
    for tr in table:
        temp=tr.find_all('td')
        try:
            date=re.findall(date_re,temp[1].text)[0]
        except:
            print(temp)
        day=int(date[0])
        month=int(date[1])
        year=int(date[2])
        rate=float(temp[3].text.split(' ')[0])
        exchange_data.append([datetime.datetime(year,month,day).date(),rate])
    headings=['Date','Rate']
    exchange_pd=pd.DataFrame(data=exchange_data,columns=headings)
    maxdate=exchange_pd['Date'].max()
    if end>maxdate:
        while maxdate+datetime.timedelta(days=1)<=end:
            pri_rate=exchange_pd[exchange_pd['Date']==maxdate]['Rate'].tolist()[0]
            temp_pd=pd.DataFrame(data=[[maxdate+datetime.timedelta(days=1),pri_rate]],columns=headings)
            exchange_pd=pd.concat([exchange_pd,temp_pd]).reset_index(drop = True)
            maxdate+=datetime.timedelta(days=1)
    return exchange_pd

In [7]:
months_lookup={'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,
              'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
def st_conversion(row):
    Mth,day=row['Date'].split(' ')
    Mth=months_lookup[Mth]
    day=int(day)
    row['Timestamp']=datetime.datetime(row['Year'],Mth,day)
    return row

def get_holiday():
    headings=['Day', 'Date', 'Holiday Name', 'Type', 'Comments','Year']
    holiday_data=pd.DataFrame()
    today=datetime.date.today()
    cur_year=today.year
    prior_year=cur_year-1
    year_list=[prior_year,cur_year]
    
    for year in year_list:
        url='https://www.officeholidays.com/countries/mexico/'+str(year)
        html_content = requests.get(url).text
        soup = BeautifulSoup(html_content, "lxml")
        H_table_obj = soup.find("table", attrs={"class": "country-table"})
        table=H_table_obj.tbody.find_all("tr")

        # Get all the rows of table
        table_data = []
        for tr in table: # find holiday_data.iloc[1]['Date'].split(' ')all tr's from table's tbody
            row=[]
            for td in tr.find_all('td'): 
                row.append(td.text.replace('\n', '').strip())
            row.append(year)
            table_data.append(row)
        
        temp_pd=pd.DataFrame(data=table_data,columns=headings)
        holiday_data=pd.concat([holiday_data,temp_pd])
    
    holiday_data['Timestamp']=None
    holiday_data=holiday_data.apply(st_conversion,axis=1)
    return holiday_data



In [8]:


def daily_weather_scrapper(day_num,city):
    city=city.lower()
    latitude=str(location[city]['lat'])
    longitude=str(location[city]['lon'])
    units='standard' #return temperature in kelvin
    appid = "b5100260400cfb368667e820bb4b58f0"
    day = requests.get("https://api.openweathermap.org/data/2.5/onecall/"\
                        +"timemachine?lat="+latitude\
                        +"&lon="+longitude\
                        +"&units="+units\
                        +"&dt="+str(day_num)\
                        +"&appid="+appid)
    day_scrapped = pd.DataFrame(day.json()['hourly'])
    day_scrapped['dt_iso'] = day_scrapped['dt'].map(lambda x:datetime.datetime.fromtimestamp(x,timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))
    day_scrapped['city_name']=city
    day_scrapped = day_scrapped[['dt','dt_iso','city_name','temp','feels_like'\
                                 ,'pressure','humidity'\
                                 ,'wind_speed','wind_deg']]
    return day_scrapped



def daily_weather_forecast(day_num,city):
    city=city.lower()
    latitude=str(location[city]['lat'])
    longitude=str(location[city]['lon'])
    units='standard' #return temperature in kelvin
    appid = "b5100260400cfb368667e820bb4b58f0"
    day = requests.get("https://api.openweathermap.org/data/2.5/onecall?"\
                        +"lat="+latitude\
                        +"&lon="+longitude\
                        +"&exclude=minutely,daily&appid=b5100260400cfb368667e820bb4b58f0")
    day_scrapped = pd.DataFrame(day.json()['hourly'])
    day_scrapped['dt_iso'] = day_scrapped['dt'].map(lambda x:datetime.datetime.fromtimestamp(x,timezone.utc).strftime('%Y-%m-%d %H:%M:%S'))
    day_scrapped['city_name']=city
    day_scrapped = day_scrapped[['dt','dt_iso','city_name','temp','feels_like'\
                                 ,'pressure','humidity'\
                                 ,'wind_speed','wind_deg']]
    return day_scrapped



def get_weather():
    ahora = int(time.time()) #current unix day
    hora_previa = ahora - (ahora % (60*60*24));#-18000;
    # datetime.utcfromtimestamp(hora_previa).strftime('%Y-%m-%d %H:%M:%S')
    days1 = hora_previa - (1*(60*60*24))
    days2 = hora_previa - (2*(60*60*24))
    days3 = hora_previa - (3*(60*60*24))
    days4 = hora_previa - (4*(60*60*24))
    days5 = hora_previa - (5*(60*60*24))
    days_list = [days5, days4, days3, days2, days1,hora_previa]
    
    historical_data=pd.read_csv('historical_weather.csv')
    historical_data=historical_data[historical_data['dt']<hora_previa]
    
    for city in location.keys():
        data_list= [daily_weather_scrapper(i,city) for i in days_list]
        f_data=daily_weather_forecast(hora_previa,city)
        historical_data['city_name']=historical_data['city_name'].map(lambda x:x.lower())
        temp=historical_data[historical_data['city_name']==city.lower()]
        maxdate=temp['dt'].max()
        n=0
        error_msg=''
        for n,item in enumerate(days_list):
            if item>maxdate:
                break;
        else:
            error_msg='updated'
        if error_msg!='updated':
            data_list=data_list[n:]
            previous5_days = pd.concat(data_list).reset_index(drop = True)
            historical_data = pd.concat([historical_data,previous5_days]).reset_index(drop = True)
        historical_data = pd.concat([historical_data,f_data]).reset_index(drop = True)
        historical_data.to_csv('historical_weather.csv')
    return historical_data
    
    
    
    

In [9]:
def validation(dataset,timecol,hour=1,day=0):
    dataset=dataset.sort_values(timecol)
    start_date=dataset[timecol].min()
    end_date=dataset[timecol].max()
    current = start_date
    #impute missing value
#     print('missing:')
    while current<end_date:
        if len(dataset[dataset[timecol]==current])==0:
            prior=current-datetime.timedelta(hours=hour,days=day)
            temp=dataset[dataset[timecol]==prior].copy(deep=True)
#             print(current)
            temp[timecol]=current
            dataset=pd.concat([dataset,temp])
        
        current+=datetime.timedelta(hours=hour,days=day)
    dataset=dataset.reset_index(drop = True)
    dataset=dataset.sort_values(timecol)
    #remove duplicates
#     print('duplicates')
    current = start_date
    dataset['Filter']=True
    while current<end_date:
        if len(dataset[dataset[timecol]==current])>1:
#             print(current)
            temp=dataset[dataset[timecol]==current].iloc[1:]
            dataset.at[temp.index,'Filter']=False

        current+=datetime.timedelta(hours=hour,days=day)
    dataset=dataset[dataset.Filter==True].reset_index(drop = True)
    dataset=dataset.drop('Filter',axis=1)
    return dataset


#Consolidation of dataset
def consol_data(city,lmp_data,ex_data,crude_pd,weather_pd):
    lmp_data['Date']=pd.to_datetime(lmp_data['Date'])
    lmp_data['Timestamp']=pd.to_datetime(lmp_data['Timestamp'])
    lmp_hourly_data=lmp_data[lmp_data['Region']==city.upper()]
    lmp_hourly_data=lmp_hourly_data.reset_index()
    lmp_hourly_data=validation(lmp_hourly_data,'Timestamp')
    
    ex_data['Date']=pd.to_datetime(ex_data['Date'])-datetime.timedelta(days=1)
    ex_data=validation(ex_data,'Date',0,1)
    combined_pd=lmp_hourly_data.join(ex_data.set_index('Date'),on='Date',how='inner')
    combined_pd=combined_pd.reset_index()[['Date','Timestamp','Energy','Lost','Congestion','Rate']]
    
    crude_pd['Date']=pd.to_datetime(crude_pd['Date'])-datetime.timedelta(days=1)
    crude_pd=validation(crude_pd,'Date',0,1)
    combined_pd=combined_pd.join(crude_pd.set_index('Date'),on='Date',how='inner')
    
    weather_pd=weather_pd[weather_pd['city_name']==city]
    weather_pd.dt_iso=weather_pd.dt_iso.apply(lambda x:x.replace(' +0000 UTC',''))
    weather_pd.dt_iso=pd.to_datetime(weather_pd.dt_iso)
    weather_pd=validation(weather_pd,'dt_iso')
    weather_pd=weather_pd.rename(columns={'dt_iso':'Timestamp'})
    combined_pd=combined_pd.join(weather_pd.set_index('Timestamp'),on='Timestamp',how='inner')
    combined_pd['Total']=combined_pd['Energy']+combined_pd['Lost']+combined_pd['Congestion']
    output_pd=combined_pd.copy(deep=True)
    #feature engineering
    features=['Total', 'Rate', 'Ending','feels_like','pressure','humidity','wind_speed','wind_deg']
    combined_pd['Timestamp']=combined_pd['Timestamp'].map(datetime.datetime.timestamp)
    data_features=combined_pd[features]
    
    #Time features
    day=60*60*24
    week=7*day
    year=day*365.25

    data_features['Day_sin']=np.sin(combined_pd['Timestamp']*(2*np.pi/day))
    data_features['Day_cos']=np.cos(combined_pd['Timestamp']*(2*np.pi/day))
    data_features['Week_sin']=np.sin(combined_pd['Timestamp']*(2*np.pi/week))
    data_features['Week_cos']=np.cos(combined_pd['Timestamp']*(2*np.pi/week))

    # Feature Engineering Wind Velocity Vectors
    ws = data_features.pop('wind_speed')

    # Convert to radians.
    wd_rad = data_features.pop('wind_deg')*np.pi / 180

    # Calculate the wind x and y components.
    data_features['Wx'] = ws*np.cos(wd_rad)
    data_features['Wy'] = ws*np.sin(wd_rad)
    
    # Normalization
    for feature in norm_data[city]['mean'].keys():
        data_features[feature]=data_features[feature].map(lambda x:(x-norm_data[city]['mean'][feature])/norm_data[city]['std'][feature])
    
    output_pd=output_pd[['Timestamp','Total', 'Rate', 'Ending','feels_like']]
    
    return data_features[-168*2:],output_pd[-168*2:]

In [10]:
def dataget():
    today = datetime.date.today()-datetime.timedelta(days=1)
    start = today-datetime.timedelta(days=13)
    #scrap electricity data
    geteprice()
    elec_price=generate_pricepd()
    maxdate=elec_price['Date'].max()+datetime.timedelta(days=1)
    maxdate=maxdate
    #scrap crude oil price
    crude_price=get_crude(maxdate)
    
    #scrap exchange rate
    exchange_rate=get_exchange(maxdate)
    
#     #scrap holiday data
#     holiday_info=get_holiday()
    
    #scrap weather data
    weather_info=get_weather()
    
    return elec_price,crude_price,exchange_rate,weather_info

In [11]:
elec_price,crude_price,exchange_rate,weather_info=dataget()

# #consolidation
dataset,output_set=consol_data('monterrey',elec_price,exchange_rate,crude_price,weather_info)

mkdir: cannot create directory ‘data’: File exists


<ipython-input-9-b96341fd142c>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_features['Day_sin']=np.sin(combined_pd['Timestamp']*(2*np.pi/day))
<ipython-input-9-b96341fd142c>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_features['Day_cos']=np.cos(combined_pd['Timestamp']*(2*np.pi/day))
<ipython-input-9-b96341fd142c>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [12]:
#Model Architecture

In [13]:
OUT_STEPS = 168

In [14]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(12)
feedback_model = FeedBack(units=680, out_steps=OUT_STEPS)
def warmup(self, inputs):
  # inputs.shape => (batch, time, features)
  # x.shape => (batch, lstm_units)
  x, *state = self.lstm_rnn(inputs)

  # predictions.shape => (batch, features)
  prediction = self.dense(x)
  return prediction, state

FeedBack.warmup = warmup
def call(self, inputs, training=None):
  # Use a TensorArray to capture dynamically unrolled outputs.
  predictions = []
  # Initialize the lstm state
  prediction, state = self.warmup(inputs)

  # Insert the first prediction
  predictions.append(prediction)

  # Run the rest of the prediction steps
  for n in range(1, self.out_steps):
    # Use the last prediction as input.
    x = prediction
    # Execute one lstm step.
    x, state = self.lstm_cell(x, states=state,
                              training=training)
    # Convert the lstm output to a prediction.
    prediction = self.dense(x)
    # Add the prediction to the output
    predictions.append(prediction)

  # predictions.shape => (time, batch, features)
  predictions = tf.stack(predictions)
  # predictions.shape => (batch, time, features)
  predictions = tf.transpose(predictions, [1, 0, 2])
  return predictions

FeedBack.call = call
feedback_model.load_weights('./Models/HLMP_AutoRegress')

In [15]:
test=tf.expand_dims(tf.convert_to_tensor(dataset),axis=0)

In [16]:
predictions=feedback_model.predict(test)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [17]:
result=tf.squeeze(predictions,axis=0)

In [18]:
maxi=output_set['Timestamp'].max()

In [19]:
dates=[maxi+datetime.timedelta(hours=x+1) for x in range(168)]

In [20]:
output_list=['Total','Rate','Ending','feels_like']
try:
    !mkdir prediction
except:
    pass 
for n,feature in enumerate(output_list):
    data=[]
    subdata=result[:,n].numpy()*norm_data['monterrey']['std'][feature]+norm_data['monterrey']['mean'][feature]
    for i,date in enumerate(dates):
        data.append([date,subdata[i]])
    temp=pd.DataFrame(data=data,columns=['Timestamp',feature])
    title='./prediction/monterrey_'+feature+'_P.csv'
    temp.to_csv(title)
    title='./prediction/monterrey_'+feature+'_A.csv'
    temp=output_set[['Timestamp',feature]].to_csv(title)

mkdir: cannot create directory ‘prediction’: File exists
